In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def process(edl_constructor, dataframe: pd.DataFrame, pzc_she: float):
    """
    Process Monteiro data and calculate phi0-phi'
    """
    dataframe["E_SHE"] = dataframe["E vs. RHE"] - dataframe["pH"] * 59e-3
    dataframe["phi0"] = dataframe["E_SHE"] - pzc_she

    # Find unique log concentration values
    unique_logc_values = dataframe.drop_duplicates(subset="log c")["log c"].values

    # One computation for each concentration value
    processed_dfs = []

    for logc in unique_logc_values:
        df_select_logc = dataframe[dataframe["log c"] == logc].copy()
        model = edl_constructor(10**logc)

        potentials_vs_pzc = np.linspace(np.min(df_select_logc["phi0"]), 0, 50)
        sweep = model.potential_sweep(potentials_vs_pzc, tol=1e-4)

        # Interpolate the result for the phi0 values already in the dataframe
        interpolator = interp1d(potentials_vs_pzc, sweep["phi_rp"])
        df_select_logc["phi_rp"] = interpolator(df_select_logc["phi0"])
        processed_dfs.append(df_select_logc)

    return pd.concat(processed_dfs)

In [ ]:
# for species in species_dict.keys():
#     df_list = []
#     for ph in [11, 13]:
#         df = pd.read_csv(f"data/{species}_ph_{ph}_full.csv", names=['log c', 'log j', 'E vs. RHE'], header=1)
#         df['pH'] = ph
#         df_list.append(df)
#     new_df = pd.concat(df_list, ignore_index=True)
#     new_df.to_csv(f'data/{species}.csv', index=False)

In [ ]:
from edl.models import ExplicitStern as model1
from edl.models import DoubleLayerModel as model2
from edl.constants import AU_PZC_SHE_V as PZC
species_dict = {'Li': (7, 'black'), 'Na': (6, 'blue'), 'K': (5, 'red')}
ph_fillstyle = {11: 'none', 13: 'full'}
x2_dict = {'Cs': 3.5e-10, 'K': 4.1e-10, 'Na': 5.2e-10, 'Li': 5.8e-10}

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

fig, ax = plt.subplots()

for species, properties in species_dict.items():
    df = pd.read_csv(f"data/{species}.csv")
    # df = process(lambda c: model1(c, properties[0], 2, x2_dict[species]), df, pzc_she=PZC)
    df = process(lambda c: model2(c, properties[0], 2), df, pzc_she=PZC)
    for j, row in df.iterrows():
        ax.plot(
            row['phi0'] - row['phi_rp'],
            row['log j'],
            marker="o",
            fillstyle=ph_fillstyle[row['pH']],
            markersize=5,
            markeredgewidth=0.5,
            linestyle="None",
            color=species_dict[species][1])

plt.show()